# 2022 Qualifying Stats

In [3]:
import pandas as pd
import sqlite3

con = sqlite3.connect("../data/f1data.sqlite")

DRIVER_PAIRS = {
  "HAM": "RUS",
  "VER": "PER",
  "SAI": "LEC",
  "NOR": "RIC",
  "ALO": "OCO",
  "GAS": "TSU",
  "VET": "STR",
  "LAT": "ALB",
  "BOT": "ZHO",
  "MSC": "MAG",
}

# It's necessary to suffix the timestamps with `'00:0'` so they're correctly
# parsed by `to_timedelta`.

query = """
SELECT
  r.date,
  r.round,
  r.name AS gp_name,
  d.code,
  d.forename || ' ' || d.surname AS name,
  q.position,
  '00:0' || q.q1 AS q1,
  '00:0' || q.q2 AS q2,
  '00:0' || q.q3 AS q3,
  '00:0' || CASE
    WHEN q.q3 IS NULL AND q.q2 IS NULL THEN q1
    WHEN q.q3 IS NULL THEN q2
    ELSE q3
  END AS final_lap
FROM qualifying AS q
JOIN races r on q.raceId = r.raceId
JOIN drivers d on q.driverId = d.driverId
WHERE r.year = 2022
"""

quali_df = pd.read_sql(
  query,
  con,
  index_col="date",
  parse_dates="date"
)


quali_df["final_lap"] = pd.to_timedelta(quali_df["final_lap"], errors="coerce")
quali_df["q1"] = pd.to_timedelta(quali_df["q1"], errors="coerce")
quali_df["q2"] = pd.to_timedelta(quali_df["q2"], errors="coerce")
quali_df["q3"] = pd.to_timedelta(quali_df["q3"], errors="coerce")

## Pole Positions

In [4]:
poles = quali_df[quali_df["position"] == 1]
poles.set_index("round", inplace=True)
poles.index.rename("#", inplace=True)
poles[["gp_name", "code", "name", "q3"]].rename(columns={
  "gp_name": "Race",
  "code": "ID",
  "name": "Driver",
  "q3": "Pole Time",
})

,Race,ID,Driver,Pole Time
#,,,,
1,Bahrain Grand Prix,LEC,Charles Leclerc,0 days 00:01:30.558000
2,Saudi Arabian Grand Prix,PER,Sergio Pérez,0 days 00:01:28.200000
3,Australian Grand Prix,LEC,Charles Leclerc,0 days 00:01:17.868000
4,Emilia Romagna Grand Prix,VER,Max Verstappen,0 days 00:01:27.999000
5,Miami Grand Prix,LEC,Charles Leclerc,0 days 00:01:28.796000
6,Spanish Grand Prix,LEC,Charles Leclerc,0 days 00:01:18.750000
7,Monaco Grand Prix,LEC,Charles Leclerc,0 days 00:01:11.376000


## Average Qualifying Time in Sessions

This approach simply takes the average of the available qualifying times in
each session.

It's basically useless, since `NaT`s in the data tend to lower the average and
heavily distort the results. I believe this is mostly due to having different
circuits.

In [5]:
avg_quali_df = quali_df[["code", "q1", "q2", "q3"]].groupby("code").mean()
avg_quali_df.sort_values(by=["q3", "q2", "q1"], inplace=True)
avg_quali_df.index.rename("Driver", inplace=True)
avg_quali_df.rename(columns={"q1": "Q1", "q2": "Q2", "q3": "Q3"})

,Q1,Q2,Q3
Driver,,,
OCO,0 days 00:01:22.967500,0 days 00:01:22.733600,0 days 00:01:20.392000
MSC,0 days 00:01:23.981428571,0 days 00:01:23.891285714,0 days 00:01:20.638000
VET,0 days 00:01:21.291400,0 days 00:01:20.928000,0 days 00:01:21.897000
RUS,0 days 00:01:23.604571428,0 days 00:01:23.280428571,0 days 00:01:22.351600
HAM,0 days 00:01:23.797571428,0 days 00:01:22.246333333,0 days 00:01:22.352000
NOR,0 days 00:01:23.738571428,0 days 00:01:23.198571428,0 days 00:01:22.358250
RIC,0 days 00:01:23.867000,0 days 00:01:22.082500,0 days 00:01:23.023666666
SAI,0 days 00:01:23.042428571,0 days 00:01:22.598285714,0 days 00:01:23.041666666
LEC,0 days 00:01:22.870142857,0 days 00:01:22.695000,0 days 00:01:23.478714285


The only exception is that it makes it clear that some pilots didn't reach Q3
or Q2:

In [6]:
criteria = avg_quali_df["q3"].isna() | avg_quali_df["q2"].isna()
sorted = avg_quali_df[criteria].sort_values(by="q2")
sorted.index.rename("Driver", inplace=True)
sorted.rename(columns={"q1": "Q1", "q2": "Q2", "q3": "Q3"})

,Q1,Q2,Q3
Driver,,,
ZHO,0 days 00:01:24.173285714,0 days 00:01:25.209000,NaT
ALB,0 days 00:01:24.979166666,0 days 00:01:32.664000,NaT
LAT,0 days 00:01:25.205285714,NaT,NaT
HUL,0 days 00:01:31.660000,NaT,NaT
